## Cabeçalho
### Objetivo
Limpar/Tratar os dados de eventos disponíveis em tb_eventos_bronze extraídos da API de dados abertos da Câmara dos Depoutados.

**Sobre
Data de crição**: 30/09/2025

**Responsável:** Wandrieli Nery Barbosa

Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Transformação

In [0]:
%sql
select * from tb_eventos_bronze;

In [0]:
set_catalog_and_schema_silver() #Define catalog e schema conforme run

df_eventos_bronze = spark.table("tiller.bronze.tb_eventos_bronze") # Carrega a tabela bronze

# Transformações
df_eventos_silver = (
    df_eventos_bronze
    .withColumn("data", F.to_timestamp("dataHoraFim"))
    .withColumn("dataHoraInicio", F.to_timestamp("dataHoraInicio"))
    .withColumn("descricao", F.initcap(F.trim("descricao")))
    .withColumn("descricaoTipo", F.initcap(F.trim("descricaoTipo")))
    .withColumn("localExterno", F.initcap(F.trim("localExterno")))

    .withColumn("localCamara_nome", F.col("localCamara").getItem("nome"))
    .withColumn("localCamara_predio", F.col("localCamara").getItem("predio"))
    .withColumn("localCamara_sala", F.col("localCamara").getItem("sala"))
    .withColumn("localCamara_andar", F.col("localCamara").getItem("andar"))

    .withColumn("orgaos_nomePublicacao", F.col("localCamara").getItem("nomePublicacao"))
    .withColumn("orgaos_uri", F.col("localCamara").getItem("uri"))
    .withColumn("orgaos_codTipoOrgao", F.col("localCamara").getItem("codTipoOrgao"))
    .withColumn("orgaos_tipoOrgao", F.col("localCamara").getItem("tipoOrgao"))
    .withColumn("orgaos_nomeResumido", F.col("localCamara").getItem("nomeResumido"))
    .withColumn("orgaos_id", F.col("localCamara").getItem("id"))
    .withColumn("orgaos_sigla", F.col("localCamara").getItem("sigla"))
    .withColumn("orgaos_apelido", F.col("localCamara").getItem("apelido"))
    .withColumn("orgaos_nome", F.col("localCamara").getItem("nome"))


    .withColumn("localCamara_nome", F.initcap(F.trim("localCamara_nome")))
    .withColumn("localCamara_predio", F.initcap(F.trim("localCamara_predio")))
    .withColumn("localCamara_sala", F.initcap(F.trim("localCamara_sala")))

    .withColumn("orgaos_nomePublicacao", F.initcap(F.trim("orgaos_nomePublicacao")))
    .withColumn("orgaos_uri", F.initcap(F.trim("orgaos_uri")))
    .withColumn("orgaos_codTipoOrgao", F.initcap(F.trim("orgaos_codTipoOrgao")))
    .withColumn("orgaos_tipoOrgao", F.initcap(F.trim("orgaos_tipoOrgao")))
    .withColumn("orgaos_nomeResumido", F.initcap(F.trim("orgaos_nomeResumido")))
    .withColumn("orgaos_sigla", F.upper(F.trim("orgaos_sigla")))
    .withColumn("orgaos_apelido", F.initcap(F.trim("orgaos_apelido")))
    .withColumn("orgaos_nome", F.initcap(F.trim("orgaos_nome")))


    .withColumn("situacao", F.initcap(F.trim("situacao")))
    .withColumn("situacao", F.initcap(F.trim("situacao")))
    .withColumn("situacao", F.initcap(F.trim("situacao")))
    .withColumn("situacao", F.initcap(F.trim("situacao")))
    .withColumn("data_ingestao_silver", F.current_timestamp()) # adiciona coluna de controle da camada silver

    .withColumn("localCamara_nome", 
                F.when(F.col("localCamara_nome").isNull(), "Evento Externo")
                .otherwise(F.col("localCamara_nome"))
                )
    
    .withColumn("orgaos_nome", 
                F.when(F.col("orgaos_nome").isNull(), "Não informado")
                .otherwise(F.col("orgaos_nome"))
                ) 

    .withColumn("localExterno", 
                F.when(F.col("localExterno").isNull(), "Evento Interno")
                .otherwise(F.col("localExterno"))
                )       

    .drop("localCamara")
    .drop("orgaos")
    .filter(F.col("id").isNotNull())

)


In [0]:
# Salva a tabela em Delta 
df_eventos_silver.write.format("delta").mode("overwrite").saveAsTable("tb_eventos_silver")

print("Tabela criada/atualizada com sucesso!")

In [0]:
%sql
select * from tiller.silver.tb_eventos_silver;